# Hot-Pow
### Investigating SnowEx Temperature Observations at Grand Mesa, Colorado, Winter 2019-2020

#### Learning objectives
* **Data Wrangling**
    * How can we pull in new external data sources and format these for use
* **Data Visualization**
    * Timeseries plotting with matplotlib and other interactive plotters (e.g., Holoviz)
* **Advanced Geospatial Visualization**
    * GIS style and multi-dimensional plotting
* **Exploring Science Questions using Thermal Data**
    * *Add more here which gets echoed under summary* 
    * How does SnowEx data (especially temperatures) compare to the ERA5 reanalysis product? 
    * How do near surface temperatures taken at snow puts compare to other thermal datasets? and more...

In [1]:
# add flowchart for going from in-situ to airborne to model data

## Grand Mesa, Colorado, Winter 2019-2020
![foo](images/grand_mesa_snotel.png)

## Snow pit temperature data
*Add brief description....*    

![foo](images/bokeh_plot-6.png)

### Data wrangling

We considered:

* Grand Mesa IOP snow pit temperature data
* Grand Mesa Time Series snow pit temperature data
* COVID quarantine extended snow pit temperature data (which will be available with SnowEx data following QA/QC)

Our hackweek exploration (as Python/Jupyter/Git newbies): can we collect, transform, and integrate these data such that we can look at a timeline of snow temperatures on the Mesa from a winter snowpack, through warming and ripening?


# # show some screenshots of our data wrangling here

In [4]:
import pandas as pd
import os
import glob
import geopandas   

import utm  
import numpy as np
import hvplot.pandas  # noqa
import hvplot.dask  # noqa

# use glob to get all the csv files in the data folder
path = './data/'
csv_files = glob.glob(os.path.join(path, "*.csv"))
layer_df = pd.DataFrame()

def utm_to_latlon(coords, zone_number, zone_letter):
    easting = coords[0]
    northing = coords[1]
    return utm.to_latlon(easting, northing, zone_number, zone_letter)

  
# loop over the list of csv files
for f in csv_files:
    df = pd.read_csv(f, header=None)
    dfdata = pd.read_csv(f, header = 13)
    # grab the metadata only
    dfmeta = df.head(12)
    # transpose the metadata to columns
    dftranspose = dfmeta.transpose().head(2)
    # grab column names from the first row
    newheader = dftranspose.iloc[0]
    # remove the column names row
    dftranspose = dftranspose[1:]
    # add column names to transposed metadata
    dftranspose.columns = newheader
    # duplicate rows of metadata to length of snow pit temperature profile
    dfdup = dftranspose.reindex(dftranspose.index.repeat(len(dfdata))).reset_index(drop=True)    
    # merge dfdata with dfdup to make a complete df of snow profile temps
    dfmerged = pd.concat([dfdup, dfdata], axis = 1)
    del dfmerged['Hs:'] # remove the total HS measurement column
    # create snow pit location geometry
    gdf = geopandas.GeoDataFrame(
    dfmerged, geometry=geopandas.points_from_xy(dfmerged['Easting:'], dfmerged['Northing:']))

    layer_df = layer_df.append(dfmerged)


### 16 columns across X files

In [5]:
layer_df.shape

(661, 16)

## Sample few rows

In [8]:
layer_df.head()

,Location:,Type:,Easting:,Northing:,Surveyor:,Time:,Time Type:,Air Temp:,Ground:,Notes:,Wx:,Hs,Temperature,Notes,geometry,Wx:
0,Skyway,Trees,754148,4325844,J. Lund,5/3/20 18:32,PM,3.2,"Smooth, grass ~10cm",No stratigraphy. I'm assuming I thought it wou...,"Partly cloudy, light breeze.",63,-0.2,NaN,POINT (754148.000 4325844.000),NaN
1,Skyway,Trees,754148,4325844,J. Lund,5/3/20 18:32,PM,3.2,"Smooth, grass ~10cm",No stratigraphy. I'm assuming I thought it wou...,"Partly cloudy, light breeze.",60,-0.2,NaN,POINT (754148.000 4325844.000),NaN
2,Skyway,Trees,754148,4325844,J. Lund,5/3/20 18:32,PM,3.2,"Smooth, grass ~10cm",No stratigraphy. I'm assuming I thought it wou...,"Partly cloudy, light breeze.",50,-0.2,NaN,POINT (754148.000 4325844.000),NaN
3,Skyway,Trees,754148,4325844,J. Lund,5/3/20 18:32,PM,3.2,"Smooth, grass ~10cm",No stratigraphy. I'm assuming I thought it wou...,"Partly cloudy, light breeze.",40,-0.2,NaN,POINT (754148.000 4325844.000),NaN
4,Skyway,Trees,754148,4325844,J. Lund,5/3/20 18:32,PM,3.2,"Smooth, grass ~10cm",No stratigraphy. I'm assuming I thought it wou...,"Partly cloudy, light breeze.",30,-0.2,NaN,POINT (754148.000 4325844.000),NaN


### Visualization

#### Temperature profiles at Height 100 across 'Open' and 'Trees' at a particular site looking at DAY time vs NIGHT Time


In [11]:
Trees_Skyway = layer_df[(layer_df["Location:"]=="Skyway") &  (layer_df["Type:"]=="Trees") &  (layer_df["Hs"]==100)].sort_values(by='Time:').hvplot.bar(y='Temperature', x='Time Type:', height=500)
Open_Skyway  = layer_df[(layer_df["Location:"]=="Skyway") &  (layer_df["Type:"]=="Open") &  (layer_df["Hs"]==100)].sort_values(by='Time:').hvplot.bar(y='Temperature', x='Time Type:', height=500)

plot = Trees_Skyway + Open_Skyway
plot

:Layout
   .Bars.I  :Bars   [Time Type:]   (Temperature)
   .Bars.II :Bars   [Time Type:]   (Temperature)

## All the Pits at Skyway, how does it vary overtime ?

In [10]:
layer_df.hvplot.line(y='Temperature', x='Time:', groupby=['Location:','Hs'], height=500)

:DynamicMap   [Location:,Hs]
   :Curve   [Time:]   (Temperature)

#### Takeaways

* Integrating data from different sources poses different challenges
    * Reorienting and matching data frames took some troubleshooting
    * Pandas, Geopandas, and Holoviz
* A season-long integration of snow pit data can be useful to compare with other observations of snow temperatures

## Thermal IR
*Add brief description....*  

### Data wrangling
*Add description of where the data came from and what needed to be done to get it analysis ready.*   
*Feel free to just load the preprocessed file from disk, instead of going through all the preprocessing steps.*

### Visualization

### Takeaways

## ERA5 vs. SNOTEL vs. SnowEx Temperature comparison

### How does modeled ERA5 climate data compare to SNOTEL and SnowEx measurements?   

Location: Grand Mesa   
Bounds: 39.2, -108.4, 38.8, -107.7   
Time range: February 2020   
#### Some ERA5 variables of interest:

- 2m_temperature
- snow_albedo
- snow_density
- snow_depth
- snow_depth_water_equivalent
- temperature_of_snow_layer
- total_precipitation   


In this example we will pull in the [ERA5-Land hourly data from 1981 to present](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=form)  

<p align="center">
<img src="https://datastore.copernicus-climate.eu/c3s/published-forms/c3sprod/reanalysis-era5-land/overview.png" width=500>   
</p>

| DATA DESCRIPTION  |  |
| ------------- | ------------- |
| **Data type**  | Gridded |
| **Projection**  | Regular latitude-longitude grid |
| **Horizontal coverage**  | Global |
| **Horizontal resolution**  | 0.1° x 0.1°; Native resolution is 9 km. |
| **Vertical coverage**  | From 2 m above the surface level, to a soil depth of 289 cm. |
| **Vertical resolution**  | 4 levels of the ECMWF surface model: Layer 1: 0 -7cm, Layer 2: 7 -28cm, Layer 3: 28-100cm, Layer 4: 100-289cm Some parameters are defined at 2 m over the surface. |
| **Temporal coverage**  | January 1981 to present |
| **Temporal resolution**  | Hourly |
| **File format**  | GRIB / NetCDF |
| **Update frequency**  | Monthly with a delay of about three months relatively to actual date. |    

 
Additional products and documentation can be found here:   
[ERA5-Land: documentation](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation)      
[ERA5-Land monthly averaged data from 1981 to present](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land-monthly-means?tab=form)     
[ERA5 monthly averaged data on pressure levels from 1950 to 1978 (preliminary version)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels-monthly-means-preliminary-back-extension?tab=overview)    
[ERA5 hourly data on pressure levels from 1950 to 1978 (preliminary version)](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels-preliminary-back-extension?tab=overview)    


### Data wrangling
Detailed data wrangling steps are outlined in [this notebook](https://github.com/snowex-hackweek/hot-pow/blob/main/contributors/Friedrich_Knuth/ERA5_temperature_comparison.ipynb), including:  
- Pull in data from ERA5 API
- Explore metadata and unit conversion
- Quickplots to inspect query results

#### Example API request

In [5]:
# c.retrieve(
#     'reanalysis-era5-land',
#     {
#         'format': 'netcdf',
#         'variable': [
#             '2m_temperature', 'snow_albedo', 'snow_density',
#             'snow_depth', 'snow_depth_water_equivalent', 'temperature_of_snow_layer',
#             'total_precipitation',
#         ],
#         'year': '2020',
#         'month': [
#             '01', '02', '03',
#         ],
#         'day': [
#             '01', '02', '03',
#             '04', '05', '06',
#             '07', '08', '09',
#             '10', '11', '12',
#             '13', '14', '15',
#             '16', '17', '18',
#             '19', '20', '21',
#             '22', '23', '24',
#             '25', '26', '27',
#             '28', '29', '30',
#             '31',
#         ],
#         'time': [
#             '00:00', '01:00', '02:00',
#             '03:00', '04:00', '05:00',
#             '06:00', '07:00', '08:00',
#             '09:00', '10:00', '11:00',
#             '12:00', '13:00', '14:00',
#             '15:00', '16:00', '17:00',
#             '18:00', '19:00', '20:00',
#             '21:00', '22:00', '23:00',
#         ],
#         'area': [
#             39.2, -108.4, 38.8,
#             -107.7,
#         ],
#     },
#     'data/era5_2020-02-01_2020-02-29.nc')

### Visualization

#### ERA5 variables over Grand Mesa on 2020-01-31 at 18:00:00

![foo](images/era5_2020-01-31.jpeg)

#### Where is the Mesa?

![foo](images/era5_grand_mesa_2020-01-31.jpeg)

#### Where are the SNOTEL sites?

![foo](images/era5_grand_mesa__with_snotel_2020-01-31.jpeg)

### How do the ERA5 time series at SNOTEL sites compare to SNOTEL time series?
- use xarray with cubic interpolation at SNOTEL site locations. See [data wrangling notebook](https://github.com/snowex-hackweek/hot-pow/blob/main/contributors/Friedrich_Knuth/ERA5_temperature_comparison.ipynb) for more details.

![foo](images/era5_snotal_compare.jpeg)

### Takeaways

## Conopy shadows and impact on snow surface temperature
*Add brief description....*  

### Data wrangling
*Add description of where the data came from and what needed to be done to get it analysis ready.*   
*Feel free to just load the preprocessed file from disk, instead of going through all the preprocessing steps.*

### Visualization

### Takeaways

## Summary
*Summary and takeaways*